In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

import nest_asyncio
nest_asyncio.apply()

In [2]:
from IPython.display import Markdown, display

username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")
url = os.getenv("NEO4J_URI")

In [13]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

graph_store = Neo4jPropertyGraphStore(
    username=username,
    password=password,
    url=url,
)

index = PropertyGraphIndex.from_existing(
    property_graph_store=graph_store,
    # llm=OpenAI(model="gpt-4o-mini", temperature=0.3),
    # embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),

)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


In [16]:
query_engine = index.as_query_engine(include_text=False)
response = query_engine.query("What is Graph Retrieval-Augmented Generation?")
display(Markdown(f"{response}"))

Failed to write data to connection ResolvedIPv4Address(('34.124.169.171', 7687)) (ResolvedIPv4Address(('34.124.169.171', 7687)))
Failed to write data to connection IPv4Address(('174617a4.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.124.169.171', 7687)))


Empty Response

In [7]:
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
AUTH = (username, password)

with GraphDatabase.driver(url, auth=AUTH) as driver:
    driver.verify_connectivity()
    print("Connection established.")

Connection established.


In [11]:
records, summary, keys = driver.execute_query(
    "MATCH (p:Person) RETURN p LIMIT 25",
    database_="neo4j",
)

# Loop through results and do something with them
for record in records:
    print(record.data())  # obtain record as dict

# Summary information
print("The query `{query}` returned {records_count} records in {time} ms.".format(
    query=summary.query, records_count=len(records),
    time=summary.result_available_after
))

C:\Users\jayit\AppData\Local\Temp\ipykernel_33448\2049424530.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


{'p': {'description': 'School of Intelligence Science and Technology, Peking University, China', 'id': 'Boci Peng', 'embedding': [-0.0772426575422287, 0.013113635592162609, 0.0027811280451714993, 0.00581658910959959, 0.00892712827771902, -0.08452940732240677, 0.058514662086963654, -0.002844948321580887, -0.05436353385448456, 0.004521625116467476, 0.05533543974161148, -0.059715356677770615, 0.08464682847261429, -0.09873135387897491, -0.04268024489283562, 0.03146640583872795, -0.0005564935854636133, -0.09156132489442825, -0.008708812296390533, -0.15624560415744781, 0.001275766990147531, -0.015322355553507805, 0.12488394230604172, -0.025055361911654472, -0.05031707510352135, -0.002314269309863448, 0.06575346738100052, 0.005891710519790649, 0.011750889010727406, -0.01918509043753147, 0.04290127009153366, 0.073848657310009, 0.07031851261854172, 0.0012936854036524892, -0.006469309329986572, 0.0897284522652626, 0.02102472260594368, -0.01840950734913349, 0.04915313795208931, 0.0062209051102399

In [15]:
print(index.property_graph_store.get_schema_str())

Node properties:
Document {fileName: STRING, fileSize: INTEGER, fileType: STRING, status: STRING, fileSource: STRING, createdAt: LOCAL_DATE_TIME, updatedAt: LOCAL_DATE_TIME, processingTime: FLOAT, errorMessage: STRING, nodeCount: INTEGER, relationshipCount: INTEGER, model: STRING, is_cancelled: BOOLEAN, total_chunks: INTEGER, processed_chunk: INTEGER, total_pages: INTEGER, id: STRING, description: STRING}
Chunk {fileName: STRING, id: STRING, text: STRING, position: INTEGER, length: INTEGER, content_offset: INTEGER, page_number: INTEGER, embedding: LIST}
Person {id: STRING, embedding: LIST, description: STRING}
Concept {id: STRING, embedding: LIST, description: STRING}
Time period {id: STRING, embedding: LIST}
Method {id: STRING, embedding: LIST, description: STRING}
Task {id: STRING, embedding: LIST, description: STRING}
System {id: STRING, embedding: LIST, description: STRING}
Algorithm {id: STRING, embedding: LIST}
Methodology {id: STRING, embedding: LIST, description: STRING}
Techno